In [11]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import spectral_norm as SN
import torch
from IPython.display import clear_output

import os
from argparse import ArgumentParser
from collections import OrderedDict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST

import pytorch_lightning as pl
cuda = True if torch.cuda.is_available() else False

In [12]:
data = pd.read_csv("./dataset/only_cat/train.csv")
data.head()


,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [13]:
data.drop(["id", "target"],axis=1, inplace= True)

In [14]:
data = data.filter(regex="bin|month")

In [15]:
#data = data[["bin_4","bin_3"]] ## fac_col 변수 순서대로 생성됨 

In [16]:
fac_cols = data.columns.tolist()
for col in fac_cols :
    print(col , len(data[col].unique()))

bin_0 2
bin_1 2
bin_2 2
bin_3 2
bin_4 2
month 12


In [17]:
import category_encoders as ce 

In [18]:
encoder = ce.OneHotEncoder(cols=fac_cols, use_cat_names= True ,handle_unknown="ignore",)
encoder.fit(data)
data_trans = encoder.transform(data)
encoder.inverse_transform(data_trans)

/opt/conda/envs/test/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,bin_0,bin_1,bin_2,bin_3,bin_4,month
0,0,0,0,T,Y,2
1,0,1,0,T,Y,8
2,0,0,0,F,Y,2
3,0,1,0,F,Y,1
4,0,0,0,F,N,8
...,...,...,...,...,...,...
299995,0,0,0,T,N,8
299996,0,0,0,F,Y,2
299997,0,0,0,F,Y,9
299998,0,1,0,F,Y,8


In [19]:
data_trans_np = data_trans.values

In [20]:
data_trans_tensor = torch.FloatTensor(data_trans_np)

In [21]:
from copy import deepcopy
map_info = deepcopy(encoder.category_mapping)
[_dict_.update({"len" : len(_dict_["mapping"])-1}) for _dict_ in map_info]

[None, None, None, None, None, None]

In [22]:
from torch.nn.functional import gumbel_softmax as g_softmax

In [23]:
def logit2onehot(logit, map_info) :
    cur_position = 0
    result = []
    for col_info in map_info :
        cur_position += col_info["len"]
        tt = logit[:, (cur_position-col_info["len"]):cur_position]
        onehot_ = g_softmax(logits= tt , hard=True,tau=1)
        result.append(onehot_)
    return torch.cat(result,axis=1)

from torch.distributions import Categorical
def logit2dist(logit,map_info) :
    cur_position = 0
    result = []
    for col_info in map_info :
        cur_position += col_info["len"]
        tt = logit[:, (cur_position-col_info["len"]):cur_position]
        dist = Categorical(logits = tt)
        result.append(dist)
    return result


In [24]:
cur_position = 0
for col_info in map_info :
    print(col_info["len"])
    cur_position += col_info["len"]
    tt = data_trans_tensor[:, (cur_position-col_info["len"]):cur_position]
    print(g_softmax(logits= tt , hard=True,tau=1))

2
tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [1., 0.],
        [0., 1.]])
2
tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [1., 0.]])
2
tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]])
2
tensor([[1., 0.],
        [1., 0.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])
2
tensor([[0., 1.],
        [0., 1.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]])
12
tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [25]:
total_column = data_trans.columns.tolist()


In [26]:
class TabularDataModule(pl.LightningDataModule) :
    def __init__(self, data , batch_size:int = 32 , num_workers:int=3) :
        super().__init__()
        self.data = data
        self.batch_size = batch_size
        self.num_workers = num_workers

        self.dims = self.data.shape[1]
    
    def prepare_data(self,) :
        pass

    def setup(self,stage=None) :
        if stage == "fit" or state is None :
            train_length = int(len(self.data)*0.8)
            lengths = [train_length, int(len(self.data)-train_length)]
            self.train, self.val = random_split(self.data, lengths)
        
        if stage == "test" or stage is None :
            self.test = self.data   


    def train_dataloader(self):
        return DataLoader(self.train , batch_size= self.batch_size , num_workers=self.num_workers )

    def valid_dataloader(self):
        return DataLoader(self.val , batch_size= self.batch_size , num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test , batch_size= self.batch_size , num_workers=self.num_workers)



In [27]:
class Generator(nn.Module):
    def __init__(self, latent_dim, out_shape, map_info , encoder, total_column):
        super().__init__()
        self.out_shape = out_shape
        self.map_info = map_info
        self.encoder = encoder
        self.total_column = total_column
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=True),
            *block(128, 128, normalize=True),
            *block(128, 64, normalize=True),
            *block(64, 32, normalize=True),
            nn.Linear(32, self.out_shape),
        )

    def forward(self, z):
        x = self.model(z)
        # img = img.view(img.size(0), *self.out_shape)
        return x

    def inference(self,z) :
        x = self.model(z)
        x = self.logit2onehot(x)
        return self.onehot2rev(x)

    def splitbycolumn(self,z) :
        logit = self.model(z)
        cur_position = 0
        result = []
        for col_info in self.map_info :
            cur_position += col_info["len"]
            tt = logit[:, (cur_position-col_info["len"]):cur_position]
            result.append(tt)
        return result

    def logit2onehot(self,logit) :
        cur_position = 0
        result = []
        for col_info in self.map_info :
            cur_position += col_info["len"]
            tt = logit[:, (cur_position-col_info["len"]):cur_position]
            onehot_ = g_softmax(logits= tt , hard=True,tau=1)
            result.append(onehot_)
        return torch.cat(result,axis=1)
    
    def logit2dist(self,logit) :
        cur_position = 0
        result = []
        for col_info in self.map_info :
            cur_position += col_info["len"]
            tt = logit[:, (cur_position-col_info["len"]):cur_position]
            dist = Categorical(logits = tt)
            result.append(dist)
        return result

    def onehot2rev(self,tensor) :
        gene_onehot_pd = pd.DataFrame(tensor.detach().numpy(),
        columns=self.total_column)
        gene_data = self.encoder.inverse_transform(gene_onehot_pd)
        return gene_data
    


In [28]:
generator = Generator(latent_dim=30, out_shape=len(total_column),
map_info=map_info,encoder=encoder , total_column=total_column)




In [29]:
z = torch.randn(100, 30)
gene_logit = generator(z)
gene_onehot_tensor = logit2onehot(gene_logit, map_info)
gene_onehot_pd = pd.DataFrame(gene_onehot_tensor.detach().numpy(),columns=total_column)
gene_data = encoder.inverse_transform(gene_onehot_pd)
gene_data

,bin_0,bin_1,bin_2,bin_3,bin_4,month
0,0,1,1,T,Y,6
1,1,1,0,F,Y,8
2,0,1,1,F,Y,7
3,0,0,0,F,Y,12
4,1,1,0,T,Y,8
...,...,...,...,...,...,...
95,1,0,0,F,N,1
96,0,0,1,F,Y,3
97,0,1,1,T,Y,5
98,0,1,1,T,N,4


In [30]:
z = torch.randn(100, 30)
# generator.inference(z)
# generator.splitbycolumn(z)

In [31]:
[dist.entropy().mean() for dist in logit2dist(gene_logit , map_info)]

[tensor(0.6840, grad_fn=<MeanBackward0>),
 tensor(0.6928, grad_fn=<MeanBackward0>),
 tensor(0.6880, grad_fn=<MeanBackward0>),
 tensor(0.6877, grad_fn=<MeanBackward0>),
 tensor(0.6926, grad_fn=<MeanBackward0>),
 tensor(2.4769, grad_fn=<MeanBackward0>)]

In [32]:
class Discriminator(nn.Module):
    def __init__(self, out_shape):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(out_shape, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, out):
        validity = self.model(out)
        return validity

In [35]:
class GAN(pl.LightningModule): 
    def __init__(self,
                input_dim = None,
                latent_dim =100 , 
                lr: float = 0.0002,
                b1: float = 0.5,
                b2: float = 0.999,
                generator_parm:dict={},
                discriminator_parm:dict={},
                batch_size: int = 64,
                **kwargs) :
        super().__init__()
        self.save_hyperparameters()

        self.generator = Generator(latent_dim=self.hparams.latent_dim,
        out_shape=self.hparams.input_dim,**generator_parm)

        self.discriminator = Discriminator(out_shape=self.hparams.input_dim,**discriminator_parm)

        self.validation_z = torch.randn(8, self.hparams.latent_dim)

        self.example_input_array = torch.zeros(2, self.hparams.latent_dim)

    def forward(self, z):
        return self.generator(z)
    
    def inference(self,z) :
        return self.generator.inference(z)

    def adversarial_loss(self, y_hat, y):
        return F.binary_cross_entropy(y_hat, y)

    def training_step(self, batch, batch_idx, optimizer_idx):
        x = batch
        # sample noise
        z = torch.randn(x.shape[0], self.hparams.latent_dim)
        z = z.type_as(x)
        # train generator
        if optimizer_idx == 0:

            # generate images
            self.generated_imgs = self(z)

            # ground truth result (ie: all fake)
            # put on GPU because we created this tensor inside training_loop
            valid = torch.ones(x.size(0), 1)
            valid = valid.type_as(x)

            # adversarial loss is binary cross-entropy
            g_loss = self.adversarial_loss(self.discriminator(self(z)), valid)
            tqdm_dict = {'g_loss': g_loss}
            output = OrderedDict({
                'loss': g_loss,
                'progress_bar': tqdm_dict,
                'log': tqdm_dict
            })
            return output

        # train discriminator
        if optimizer_idx == 1:
            # Measure discriminator's ability to classify real from generated samples

            # how well can it label as real?
            valid = torch.ones(x.size(0), 1)
            valid = valid.type_as(x)

            real_loss = self.adversarial_loss(self.discriminator(x), valid)

            # how well can it label as fake?
            fake = torch.zeros(x.size(0), 1)
            fake = fake.type_as(x)

            fake_loss = self.adversarial_loss(
                self.discriminator(self(z).detach()), fake)

            # discriminator loss is the average of these
            d_loss = (real_loss + fake_loss) / 2
            tqdm_dict = {'d_loss': d_loss}
            output = OrderedDict({
                'loss': d_loss,
                'progress_bar': tqdm_dict,
                'log': tqdm_dict
            })
            return output

    def configure_optimizers(self):
        lr = self.hparams.lr
        b1 = self.hparams.b1
        b2 = self.hparams.b2

        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=(b1, b2))
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=(b1, b2))
        return [opt_g, opt_d], []

    def on_epoch_end(self):
        z = self.validation_z.type_as(self.generator.model[0].weight)


In [ ]:
generator_parm = dict(map_info=map_info,encoder=encoder,total_column = total_column)

In [43]:
data_trans_np

array([[1, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]])

In [44]:
dm = TabularDataModule(data_trans_np.astype(np.float32))
model = GAN(dm.size(), generator_parm = generator_parm)
trainer = pl.Trainer(gpus=0, max_epochs=5, progress_bar_refresh_rate=20)
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name          | Type          | Params | In sizes | Out sizes
-----------------------------------------------------------------------
0 | generator     | Generator     | 41.2 K | [2, 100] | [2, 22]  
1 | discriminator | Discriminator | 143 K  | ?        | ?        
-----------------------------------------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/opt/conda/envs/test/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


In [46]:
import matplotlib.pyplot as plt
model.eval()
z = torch.randn(10, model.hparams.latent_dim)
model.inference(z)

,bin_0,bin_1,bin_2,bin_3,bin_4,month
0,0,0,1,F,Y,9
1,0,0,1,F,N,7
2,1,0,0,F,Y,8
3,0,0,1,F,Y,9
4,0,0,1,T,N,9
5,1,0,0,F,Y,2
6,0,0,0,T,Y,1
7,0,0,1,F,Y,8
8,1,0,1,T,Y,8
9,0,0,1,T,N,5
